**DEPRECATED: easy export to mobile failed**

In [ ]:
!pip install -qq transformers

In [ ]:
import os.path

import torch
import torch.nn as nn
from torch.utils.mobile_optimizer import optimize_for_mobile

from transformers import AutoFeatureExtractor, ASTForAudioClassification

In [ ]:
extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
extractor

In [ ]:
model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
model

In [ ]:
class ModelProbabilities(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return torch.sigmoid(self.model(input_values=x).logits)

model_prob = ModelProbabilities(model)

In [ ]:
sampling_rate = 16000
sample_input = torch.distributions.uniform.Uniform(-10000, 10000).sample((sampling_rate * 10,))

In [ ]:
sample_features = extractor(sample_input, sampling_rate, return_tensors="pt")['input_values']
sample_features.shape

In [ ]:
model_prob(sample_features).shape

In [ ]:
traced_model = torch.jit.trace(model_prob, sample_features)
traced_model_optimized = optimize_for_mobile(traced_model)

In [ ]:
def extractor_func(inputs):
    return extractor(inputs, sampling_rate, return_tensors="pt")['input_values']

traced_extractor = torch.jit.trace(extractor_func, sample_input)

traced_extractor_optimized = optimize_for_mobile(traced_extractor)

In [ ]:
output_path = './drive/MyDrive/Projects/MiniSoundFinder_v2/exports/'

traced_model._save_for_lite_interpreter(
    os.path.join(output_path, 'ast.pt'))

traced_extractor._save_for_lite_interpreter(
    os.path.join(output_path, 'extractor.pt'))